In [0]:
from pyspark.sql.functions import col, when

df = spark.read.table("hcmut.gold.fact_vn_weather_daily")
selected_columns = [
    "dt_date_record",
    "ds_location",
    "nr_temperature_2m_mean",
    "nr_relative_humidity_2m_mean",
    "nr_precipitation_sum",
    "nr_pressure_msl_mean",
    "nr_cloud_cover_mean",
    "nr_sunshine_duration",
    "nr_wind_speed_10m_mean",
    "nr_soil_moisture_0_to_7cm_mean",
    "nr_weather_code"
]

df = df.withColumn(
    "loc_hcm",
    when(col("ds_location") == "Ho Chi Minh City", 1).otherwise(0)
).withColumn(
    "loc_dn",
    when(col("ds_location") == "Da Nang City", 1).otherwise(0)
).withColumn(
    "loc_hn",
    when(col("ds_location") == "Ha Noi City", 1).otherwise(0)
)
selected_columns.extend(["loc_hcm", "loc_dn", "loc_hn"])

df = (
    df.select(*selected_columns)
      .dropna()
      .dropDuplicates(["dt_date_record"])
      .orderBy("dt_date_record")
)

pdf = df.toPandas().sort_values("dt_date_record")
pdf = pdf.reset_index(drop=True)

rain_codes = [51,53,55,61,63,65]
pdf["rain_label"] = pdf["nr_weather_code"].apply(lambda x: 1 if x in rain_codes else 0)

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

numeric_cols = [
    "nr_temperature_2m_mean",
    "nr_relative_humidity_2m_mean",
    "nr_precipitation_sum",
    "nr_pressure_msl_mean",
    "nr_cloud_cover_mean",
    "nr_sunshine_duration",
    "nr_wind_speed_10m_mean",
    "nr_soil_moisture_0_to_7cm_mean"
]

location_cols = ["loc_hcm", "loc_dn", "loc_hn"]

lag_dfs = []
for lag in range(1, 25):
    lag_df = pdf[numeric_cols].shift(lag)
    lag_df.columns = [f"{c}_lag_{lag}" for c in numeric_cols]
    lag_dfs.append(lag_df)

pdf = pd.concat([pdf] + lag_dfs, axis=1).dropna().reset_index(drop=True)

feature_cols = [c for c in pdf.columns if any(x in c for x in numeric_cols)] + location_cols

scaler = MinMaxScaler()
X_numeric_scaled = scaler.fit_transform(pdf[[c for c in feature_cols if c not in location_cols]])
X_scaled = np.hstack([X_numeric_scaled, pdf[location_cols].values])


In [0]:
seq_len = 3
X_list = []
for i in range(seq_len, len(X_scaled)):
    X_list.append(X_scaled[i-seq_len:i])

X = np.array(X_list)
y = pdf["rain_label"].values[seq_len:]

In [0]:
split_ratio = 0.8
split_idx = int(len(X) * split_ratio)

X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(seq_len, X.shape[2])),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# --- THAY ĐỔI 1: TÍNH TOÁN CLASS WEIGHT ---
# Đếm số lượng 0 và 1
from sklearn.utils import class_weight
import numpy as np

# y_train phải được định nghĩa từ cell trước
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(zip(np.unique(y_train), weights))

print(f"Sử dụng Class Weights (để phạt nặng hơn việc đoán sai Lớp 0): {class_weights_dict}")
# Ví dụ output: {0: 2.55, 1: 0.62}

# --- THAY ĐỔI 2: THÊM `class_weight` VÀ TĂNG `batch_size` ---
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,  
    validation_split=0.1,
    callbacks=[es],
    class_weight=class_weights_dict, 
    verbose=1
)

In [0]:
from sklearn.metrics import accuracy_score, classification_report
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Reconstructing the confusion matrix from the user's classification report
# Report:
#                precision    recall  f1-score   support
#            0       0.54      0.73      0.62        83
#            1       0.93      0.84      0.88       341

# Total samples
total_actual_0 = 83
total_actual_1 = 341

# From Recall
# True Negative (TN) = Recall_0 * Total_Actual_0
tn = round(0.73 * total_actual_0) # 60.59 -> 61
# True Positive (TP) = Recall_1 * Total_Actual_1
tp = round(0.84 * total_actual_1) # 286.44 -> 286

# From totals
# False Positive (FP) = Total_Actual_0 - TN
fp = total_actual_0 - tn # 83 - 61 = 22
# False Negative (FN) = Total_Actual_1 - TP
fn = total_actual_1 - tp # 341 - 286 = 55

# Create the matrix
cm_data = [[tn, fp],
           [fn, tp]]

cm_df = pd.DataFrame(cm_data,
                     index = ['Actual 0 (No Rain)', 'Actual 1 (Rain)'], 
                     columns = ['Predicted 0 (No Rain)', 'Predicted 1 (Rain)'])

display(cm_df)

plt.figure(figsize=(8,6))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Rain Prediction (Improved Model)')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')

plot_filename = 'confusion_matrix_rain_improved.png'
plt.savefig(plot_filename)

In [0]:
import matplotlib.pyplot as plt

# Lấy 30 ngày cuối
last_1000 = pdf.tail(1000)

# Mốc thời gian cách 5 ngày 1 lần
xticks = last_1000["dt_date_record"].iloc[::30]

plt.figure(figsize=(15, 3))

# Vẽ dạng cột: có cột nếu rain_label = 1
plt.bar(last_1000["dt_date_record"], last_1000["rain_label"])

plt.xlabel("Time")
plt.ylabel("Rain (1 = Rain)")
plt.title("Rain Label - Last 1000 Days (Bar Plot)")
plt.xticks(xticks, rotation=45)

plt.tight_layout()
plt.show()


In [0]:
# --- LOGGING DỰ ĐOÁN MƯA (VỚI SIGNATURE) ---

import mlflow
import mlflow.keras
import mlflow.pyfunc
from mlflow.models.signature import infer_signature
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np

# --- 7.1. Lấy các biến đã huấn luyện từ các cell trước ---
# (Đảm bảo các biến này đã tồn tại sau khi huấn luyện)

try:
    trained_model = model   # Model đã fit (đã tinh chỉnh)
    trained_scaler = scaler # Scaler đã fit

    # Lấy các cột numeric đã được lag
    numeric_feature_cols = [c for c in feature_cols if c not in location_cols]
    
    print("Đã xác định các biến huấn luyện (model, scaler, pdf, etc.).")
    print(f"Sequence length: {seq_len}")
    print(f"Total features: {len(feature_cols)}")
    print(f"Numeric/Lagged features: {len(numeric_feature_cols)}")

except NameError as e:
    print(f"LỖI: Không tìm thấy biến cần thiết. Bạn đã chạy cell huấn luyện chưa?")
    print(f"Chi tiết: {e}")
    # dbutils.notebook.exit("Biến huấn luyện không tồn tại")


# --- 7.2. Định nghĩa Lớp Wrapper (xử lý "scale một phần") ---

class RainPredictionWrapper(mlflow.pyfunc.PythonModel):

    def __init__(self, model, scaler, seq_len, 
                 feature_columns, numeric_feature_cols, location_cols):
        self.model = model
        self.scaler = scaler
        self.seq_len = seq_len
        
        self.feature_columns = feature_columns
        self.numeric_feature_cols = numeric_feature_cols
        self.location_cols = location_cols
        
        self.n_features = len(self.feature_columns)

    def _preprocess(self, model_input_df):
        # 'model_input_df' là một Pandas DF (seq_len, n_total_features)
        
        # 1. Tách riêng 2 nhóm cột (giống hệt lúc training)
        numeric_data = model_input_df[self.numeric_feature_cols]
        location_data = model_input_df[self.location_cols].values
        
        # 2. CHỈ scale các cột numeric (đã được lag)
        scaled_numeric = self.scaler.transform(numeric_data)
        
        # 3. Kết hợp lại
        scaled_data = np.hstack([scaled_numeric, location_data])
        
        # 4. Reshape cho LSTM [samples, timesteps, features]
        # Input là (3, 203) -> Reshape thành (1, 3, 203)
        input_sequence = scaled_data.reshape(1, self.seq_len, self.n_features)
        return input_sequence

    def predict(self, context, model_input):
        # 'model_input' là một Pandas DataFrame của (seq_len, n_total_features)
        
        if model_input.shape[0] != self.seq_len:
            raise ValueError(f"Input data must have exactly {self.seq_len} rows (timesteps).")
            
        # Bước 1-4: Tiền xử lý (Tách, Scale, Hstack, Reshape)
        input_sequence = self._preprocess(model_input)
        
        # Bước 5: Dự đoán (Model trả về xác suất)
        prediction_prob = self.model.predict(input_sequence) # Shape (1, 1)
        
        # Bước 6: Hậu xử lý
        prob = float(prediction_prob[0][0])
        label = 1 if prob > 0.5 else 0
        
        # Trả về một DataFrame (yêu cầu của MLflow)
        return pd.DataFrame({
            "prediction_label": [label],
            "prediction_probability": [prob]
        })

print("Đã định nghĩa lớp 'RainPredictionWrapper'.")

# --- 7.3. Tạo Signature và Logging vào Unity Catalog ---

print("Đang tạo đối tượng wrapper...")
pyfunc_wrapper = RainPredictionWrapper(
    model=trained_model,
    scaler=trained_scaler,
    seq_len=seq_len,
    feature_columns=feature_cols,
    numeric_feature_cols=numeric_feature_cols,
    location_cols=location_cols
)

print("Đang chuẩn bị dữ liệu mẫu cho signature...")
try:
    # Lấy một mẫu input (ví dụ: seq_len hàng đầu tiên từ 'pdf')
    input_example = pdf[feature_cols].iloc[0:seq_len]
    
    if input_example.shape[0] != seq_len:
        raise ValueError(f"Dữ liệu 'pdf' không đủ {seq_len} dòng để làm mẫu.")

except NameError:
    print("LỖI: Không tìm thấy 'pdf'. Bạn phải chạy lại cell training để có 'pdf'.")
    # dbutils.notebook.exit("Cần 'pdf' để tạo signature")

print(f"Đang chạy dự đoán mẫu trên {input_example.shape[0]} dòng...")
output_example = pyfunc_wrapper.predict(context=None, model_input=input_example)

print("Đang suy ra signature từ input và output mẫu...")
signature = infer_signature(input_example, output_example)
print("--- Đã tạo signature thành công ---")


# 3. LOG VÀ ĐĂNG KÝ MÔ HÌNH (VỚI SIGNATURE)

# !!! QUAN TRỌNG: Sửa lại <CATALOG_NAME> và <SCHEMA_NAME> của bạn
UC_CATALOG_NAME = "hcmut"  # !!! THAY THẾ: Bằng Catalog của bạn
UC_SCHEMA_NAME = "gold"    # !!! THAY THẾ: Bằng Schema của bạn (ví dụ: 'gold' hoặc 'ml')
MODEL_NAME = "weatherforecast_rain_lstm" # Tên mô hình dự đoán mưa
model_registry_name = f"{UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.{MODEL_NAME}"

# Set Model Registry về UC
mlflow.set_registry_uri("databricks-uc")

# Đặt experiment
experiment_name = f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/LSTM_Rain_Forecast"
mlflow.set_experiment(experiment_name)

print(f"Bắt đầu logging vào Experiment: {experiment_name}")
print(f"Đăng ký mô hình UC với tên: {model_registry_name}")

with mlflow.start_run(run_name="LSTM_Rain_Forecast_UC_Run") as run:
    
    mlflow.log_param("seq_len", seq_len)
    mlflow.log_param("feature_columns_count", len(feature_cols))

    # Log các chỉ số (từ cell đánh giá cuối)
    if 'y_test' in locals():
        from sklearn.metrics import recall_score, precision_score, f1_score
        y_pred = (model.predict(X_test) > 0.5).astype(int)
        mlflow.log_metric("recall_class_0", recall_score(y_test, y_pred, pos_label=0))
        mlflow.log_metric("recall_class_1", recall_score(y_test, y_pred, pos_label=1))
        mlflow.log_metric("macro_f1", f1_score(y_test, y_pred, average='macro'))

    print("Đang log mô hình với signature...")
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=pyfunc_wrapper,
        registered_model_name=model_registry_name,
        signature=signature
    )
    
    print(f"--- HOÀN TẤT LOGGING (với Signature) ---")
    print(f"Run ID: {run.info.run_id}")
    print(f"Đã đăng ký phiên bản mới cho mô hình: '{model_registry_name}'")

In [0]:
# ---  PIPELINE DỰ BÁO MƯA HÀNG NGÀY ---
# Code này đã bao gồm TẤT CẢ các import, class, và logic

# === CELL 1: IMPORTS ===
import mlflow
import mlflow.pyfunc  # Quan trọng, phải import trước khi định nghĩa class
import pandas as pd
import numpy as np
from pyspark.sql.functions import col, when
import datetime

# === CELL 2: ĐỊNH NGHĨA CLASS (PHẢI CÓ) ===
# (Class này phải được định nghĩa để mlflow.load_model() hoạt động)

class RainPredictionWrapper(mlflow.pyfunc.PythonModel):

    def __init__(self, model, scaler, seq_len, 
                 feature_columns, numeric_feature_cols, location_cols):
        self.model = model
        self.scaler = scaler
        self.seq_len = seq_len
        self.feature_columns = feature_columns
        self.numeric_feature_cols = numeric_feature_cols
        self.location_cols = location_cols
        self.n_features = len(self.feature_columns)

    def _preprocess(self, model_input_df):
        # 1. Tách riêng 2 nhóm cột (giống hệt lúc training)
        numeric_data = model_input_df[self.numeric_feature_cols]
        location_data = model_input_df[self.location_cols].values
        
        # 2. CHỈ scale các cột numeric (đã được lag)
        scaled_numeric = self.scaler.transform(numeric_data)
        
        # 3. Kết hợp lại
        scaled_data = np.hstack([scaled_numeric, location_data])
        
        # 4. Reshape cho LSTM [samples, timesteps, features]
        input_sequence = scaled_data.reshape(1, self.seq_len, self.n_features)
        return input_sequence

    def predict(self, context, model_input):
        if model_input.shape[0] != self.seq_len:
            raise ValueError(f"Input data must have exactly {self.seq_len} rows (timesteps).")
            
        input_sequence = self._preprocess(model_input)
        prediction_prob = self.model.predict(input_sequence) # Shape (1, 1)
        
        prob = float(prediction_prob[0][0])
        label = 1 if prob > 0.5 else 0
        
        return pd.DataFrame({
            "prediction_label": [label],
            "prediction_probability": [prob]
        })

print("--- Cell 1 & 2: Imports và Class Definition đã sẵn sàng ---")


# === CELL 3: THIẾT LẬP CẤU HÌNH PIPELINE ===
print("\n--- Cell 3: Thiết lập Cấu hình Pipeline ---")

# !!! QUAN TRỌNG: Sửa lại <CATALOG_NAME> và <SCHEMA_NAME> của bạn
UC_CATALOG_NAME = "hcmut"
UC_SCHEMA_NAME = "gold"
# (Đảm bảo tên này viết thường khớp với Bước 7)
MODEL_NAME = "weatherforecast_rain_lstm" 
# (Đảm bảo alias này khớp với bạn đã đặt trong UI)
MODEL_ALIAS = "rain"                   
MODEL_REGISTRY_NAME = f"{UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.{MODEL_NAME}"

SOURCE_TABLE = "hcmut.gold.fact_vn_weather_daily" 
TARGET_TABLE = "hcmut.gold.lstm_rain_daily"

CITIES_TO_FORECAST = ["Ho Chi Minh City", "Da Nang City", "Ha Noi City"]
CITY_LOC_COLS = {"Ho Chi Minh City": "loc_hcm", "Da Nang City": "loc_dn", "Ha Noi City": "loc_hn"}

N_LAG_DAYS = 24 
SEQ_LEN = 3     

numeric_cols = [
    "nr_temperature_2m_mean", "nr_relative_humidity_2m_mean", "nr_precipitation_sum",
    "nr_pressure_msl_mean", "nr_cloud_cover_mean", "nr_sunshine_duration",
    "nr_wind_speed_10m_mean", "nr_soil_moisture_0_to_7cm_mean"
]
location_cols = ["loc_hcm", "loc_dn", "loc_hn"]
BASE_COLUMNS_TO_SELECT = ["dt_date_record", "ds_location"] + numeric_cols

# === CELL 4: TẢI MÔ HÌNH (PHỤ THUỘC CELL 1 & 2) ===
print("\n--- Cell 4: Tải Mô hình từ Registry ---")

# (FIX 1) Đặt registry về UC để chạy độc lập
mlflow.set_registry_uri("databricks-uc")

print(f"Đang tải mô hình '{MODEL_REGISTRY_NAME}' với bí danh (alias) '@{MODEL_ALIAS}'...")
model_uri = f"models:/{MODEL_REGISTRY_NAME}@{MODEL_ALIAS}"

try:
    loaded_model = mlflow.pyfunc.load_model(model_uri)
    print("--- Tải mô hình thành công ---")
except Exception as e:
    print(f"LỖI: Không thể tải mô hình. Tên model hoặc alias không đúng.")
    print(f"Model: {MODEL_REGISTRY_NAME}, Alias: @{MODEL_ALIAS}")
    print(f"Chi tiết lỗi: {e}")
    dbutils.notebook.exit(f"Model loading failed. Check alias '@{MODEL_ALIAS}'.")

# === CELL 5: LẤY DỮ LIỆU VÀ FEATURE ENGINEERING ===
print("\n--- Cell 5: Lấy dữ liệu và Feature Engineering ---")

days_to_fetch = N_LAG_DAYS + SEQ_LEN + 10 

print(f"Đang lấy {days_to_fetch} ngày dữ liệu mới nhất từ {SOURCE_TABLE}...")
pdf = (
    spark.read.table(SOURCE_TABLE)
    .select(*BASE_COLUMNS_TO_SELECT)
    .orderBy(col("dt_date_record").desc())
    .limit(days_to_fetch * 3) # (3 thành phố)
    .orderBy(col("dt_date_record").asc())
    .toPandas()
)

# 1. Tái tạo One-Hot
pdf["loc_hcm"] = (pdf["ds_location"] == "Ho Chi Minh City").astype(int)
pdf["loc_dn"] = (pdf["ds_location"] == "Da Nang City").astype(int)
pdf["loc_hn"] = (pdf["ds_location"] == "Ha Noi City").astype(int)

# 2. Tái tạo Lags
print(f"Đang tạo {N_LAG_DAYS} ngày lags...")
lag_dfs = []
for lag in range(1, N_LAG_DAYS + 1):
    lag_df = pdf.groupby("ds_location")[numeric_cols].shift(lag)
    lag_df.columns = [f"{c}_lag_{lag}" for c in numeric_cols]
    lag_dfs.append(lag_df)

pdf = pd.concat([pdf] + lag_dfs, axis=1)
pdf = pdf.dropna().reset_index(drop=True)

# 3. Định nghĩa Feature Columns
# (FIX 2) Bao gồm cả 8 cột gốc (lag 0)
numeric_feature_cols = [c for c in pdf.columns if any(x in c for x in numeric_cols)]
feature_cols = numeric_feature_cols + location_cols

print(f"Đã tạo xong {len(feature_cols)} features (bao gồm 8 cột gốc + 192 cột lag + 3 cột location).")

# === CELL 6: DỰ BÁO VÀ LƯU KẾT QUẢ ===
print("\n--- Cell 6: Thực hiện Dự báo và Lưu kết quả ---")

all_forecasts = []
today_run_time = datetime.datetime.now(datetime.timezone.utc)
forecast_date = pdf["dt_date_record"].max() + pd.Timedelta(days=1)


for city_name, loc_col in CITY_LOC_COLS.items():
    print(f"\n--- Đang xử lý cho: {city_name} ---")
    
    try:
        city_pdf = pdf[pdf["ds_location"] == city_name].copy()
        input_data = city_pdf.tail(SEQ_LEN)[feature_cols]
        
        if input_data.shape[0] != SEQ_LEN:
            print(f"LỖI: Không đủ dữ liệu (cần {SEQ_LEN} ngày) để dự báo cho {city_name}. Bỏ qua...")
            continue
        
        # (FIX 3) Ép kiểu dữ liệu để khớp với Signature
        try:
            for col_name in location_cols:
                input_data[col_name] = input_data[col_name].astype(np.int32)
            
            for col_name in numeric_feature_cols:
                input_data[col_name] = input_data[col_name].astype(np.float32)
                
        except Exception as e:
            print(f"LỖI khi ép kiểu dữ liệu: {e}")
            continue

        # Gọi mô hình
        print(f"Đang dự báo cho {city_name}...")
        forecast_result_df = loaded_model.predict(input_data)
        
        forecast_result_df['dt_forecast_date'] = forecast_date
        forecast_result_df['ds_location'] = city_name
        forecast_result_df['dt_model_run_time'] = today_run_time
        forecast_result_df['ds_model_version_uri'] = model_uri
        
        all_forecasts.append(forecast_result_df)

    except Exception as e:
        print(f"LỖI trong quá trình xử lý cho {city_name}: {e}")

# (Lưu kết quả)
if not all_forecasts:
    print("\nKhông có kết quả dự báo nào được tạo ra. Kết thúc.")
else:
    print(f"\n--- TỔNG HỢP KẾT QUẢ DỰ BÁO ({len(all_forecasts)} thành phố) ---")
    final_results_df = pd.concat(all_forecasts)
    display(final_results_df)

    print(f"Đang lưu tất cả kết quả vào bảng {TARGET_TABLE}...")
    
    try:
        results_spark_df = spark.createDataFrame(final_results_df)
        (results_spark_df.write
            .format("delta")
            .mode("append")
            .option("mergeSchema", "true") 
            .saveAsTable(TARGET_TABLE)
        )
        print("--- LƯU KẾT QUẢ VÀO DELTA LAKE THÀNH CÔNG ---")
    except Exception as e:
        print(f"LỖI khi lưu vào Delta Lake: {e}")